# Importing Libariries

In [1]:
import os
import pickle
import pandas as pd
import numpy as np

# Processing the data

In [3]:
data_path = 'WESAD/WESAD'
subjects = [f for f in os.listdir(data_path) if f.startswith('S')]
data_frames = []

for subject in subjects:
    subject_path = os.path.join(data_path, subject, f'{subject}.pkl')
    with open(subject_path, 'rb') as file:
        data = pickle.load(file, encoding='latin1')

    chest_data = data['signal']['chest']
    wrist_data = data['signal']['wrist']
    stress = data['label']

    chest_sensor_dfs = []
    for sensor_name, sensor_values in chest_data.items():
        sensor_array = np.array(sensor_values)
        if sensor_array.ndim == 1:
            sensor_array = sensor_array.reshape(-1, 1)
        num_axes = sensor_array.shape[1]
        col_names = [f'chest_{sensor_name}_{i}' for i in range(num_axes)]
        chest_sensor_df = pd.DataFrame(sensor_array, columns=col_names)
        chest_sensor_dfs.append(chest_sensor_df)
    chest_df = pd.concat(chest_sensor_dfs, axis=1)

    wrist_sensor_dfs = []
    for sensor_name, sensor_values in wrist_data.items():
        sensor_array = np.array(sensor_values)
        if sensor_array.ndim == 1:
            sensor_array = sensor_array.reshape(-1, 1)
        num_axes = sensor_array.shape[1]
        col_names = [f'wrist_{sensor_name}_{i}' for i in range(num_axes)]
        wrist_sensor_df = pd.DataFrame(sensor_array, columns=col_names)
        wrist_sensor_dfs.append(wrist_sensor_df)
    wrist_df = pd.concat(wrist_sensor_dfs, axis=1)

    subject_df = pd.concat([chest_df, wrist_df], axis=1)
    subject_df['stress'] = stress
    subject_df['subject'] = subject
    subject_df = subject_df.head(10000)
    data_frames.append(subject_df)

data_df = pd.concat(data_frames, ignore_index=True)
data_df.to_csv('data.csv', index=False)